In [4]:
from cim_optimizer.solve_Ising import *
from src.envs.utils import GraphDataset
from cim_optimizer.CIM_helper import brute_force
from cim_optimizer.solve_Ising import *
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import networkx as nx
                            


In [5]:
from cim_optimizer.optimal_params import maxcut_100_params
from cim_optimizer.optimal_params import maxcut_200_params
from cim_optimizer.optimal_params import maxcut_500_params

In [6]:
def compute_cut(matrix,spins):
  return (1/4) * np.sum( np.multiply(matrix, 1 - np.outer(spins,spins)))

def load_pickle(file_path):
  with open(file_path, 'rb') as f:
      data = pickle.load(f)
  return data

In [27]:
def solve(graph):
    # CAC hyperparameters
    p= 0.9
    alpha= 1.1
    beta= 0.35
    gamma= 0.0005
    delta= 15
    mu= 0.7
    rho= 1
    tau= 200
    noise = 0.00

    # additional run information
    num_trials = 50
    # time_span = 25000
    time_span = 12000
    nsub = 0.02
    num_parallel_runs=50

    result = Ising(-graph).solve(num_runs = num_trials, 
                                num_timesteps_per_run = time_span, 
                                num_parallel_runs=num_parallel_runs, 
                                return_spin_trajectories_all_runs=False,
                                amplitude_control_scheme=False,
                                cac_time_step=nsub, 
                                cac_r=p, 
                                cac_alpha=alpha, 
                                cac_beta=beta, 
                                cac_gamma=gamma, 
                                cac_delta=delta,
                                cac_mu=mu,
                                cac_rho=rho,
                                cac_tau=tau,
                                suppress_statements = True,
                                hyperparameters_autotune = True,
                                hyperparameters_randomtune = False,
                                use_GPU = True,
                                use_CAC =True)
    
    spins=result.result['lowest_energy_spin_config']
    cut= (1/4) * np.sum( np.multiply(graph, 1 - np.outer(spins, spins) ) )
    return cut
    # print(cut)
    
    



def OPT(distribution):
    dataset=GraphDataset(folder_path=f'../data/testing/{distribution}',ordered=True)

    try:
        OPT = load_pickle(f'../data/testing/{distribution}/optimal', add_data_path=False)['OPT']
    except:
        OPT= None

    # results={"Cut":[]}
    cuts=[]

    for _ in range(len(dataset)):
        cut=solve(dataset.get())
        cuts.append(cut)
        # break
    # print(cut)
    if OPT:
        for i,best_cut in enumerate(cuts):
            if best_cut>OPT.iloc[i]:
                OPT.iloc[i]=best_cut

    else:
        df={'OPT':cuts}
        df=pd.DataFrame(df)
        df.to_pickle(f'../data/testing/{distribution}/optimal')



OPT(distribution="WattsStrogatz_200vertices_unweighted")   


No External Field Detected
Tuned parameters: {'T_time': 12000, 'time_step': 0.02, 'gamma': 0.0005, 'delta': 15, 'mu': 0.7, 'tau': 200, 'noise': 0, 'H0': None, 'stop_when_solved': False, 'num_sol': 10, 'custom_fb_schedule': None, 'custom_pump_schedule': None, 'device': device(type='cuda'), 'batch_size': 50, 'cac_nonlinearity': <built-in method tanh of type object at 0x7f9d1ca63540>, 'beta': 0.40523292990393067, 'r': 0.7866540572118124, 'alpha': 1.269294542023794, 'rho': 1.0829071726742592}.


In [25]:
cut

434.0

In [21]:
result.result

{'lowest_energy': -799.0,
 'lowest_energy_spin_config': array([-1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        -1., -1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1.,
         1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,
         1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1.,
         1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1.,
         1., -1., -1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,
        -1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,  1.,
         1., -1.,  1., -1., -1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,
        -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,
        -1., -1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,
        -1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1.,
         1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  

In [22]:
spins=result.result['lowest_energy_spin_config']
cut= (1/4) * np.sum( np.multiply( data, 1 - np.outer(spins, spins) ) )
print(cut)

434.0


In [36]:
np.outer(spins, spins)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [33]:
problem.solver.CIM_CAC_GPU()

TypeError: Ising.solver.CIM_CAC_GPU() missing 1 required positional argument: 'self'